In [1]:
import re, os, pandas
from pingouin import pairwise_gameshowell
import pingouin as pg
import scipy.stats as stats


In [ ]:
def os_path(list) : 
    os_path = os.path.dirname(os.path.dirname(os.getcwd()))
    for i in list : 
        os_path = os.path.join(os_path, i)
    return os_path

In [2]:
df = pandas.read_csv(os_path(["database_clean", "database_clean.csv"]))


In [3]:
df.columns

Index(['Profession_classification', 'company_name', 'work_name',
       'requirements', '104_url', 'area', 'education', 'industry', 'seniority',
       'number_of_applicants', 'salary', 'company_size', 'area_district',
       'number_of_applicants_level', 'education_specialist', 'education_none',
       'education_University', 'education_high_school',
       'education_low_high_school', 'education_phd', 'education_master',
       'month_salary_classfication', 'year_salary_classfication',
       'time_salary_classfication', 'element_salary_classfication',
       'max_salary', 'mix_salary', 'company_area',
       'Profession_classification_chinese'],
      dtype='object')

# 假設變異數不同質-Anova檢定


In [3]:
# 記得改縣市別
df_base_Welchs_anova_test = pandas.DataFrame([])
df_base_Games_Howell = pandas.DataFrame([])
for Profession_classification in list(dict.fromkeys(df["Profession_classification_chinese"])) : 
    sign_list = []
    for area in list(dict.fromkeys(df["area"])): 
        if area == "台北市" : 
            continue
        sign_list.append({"縣市" : area, "職業類型" :Profession_classification})
        
    # welch_anova 檢定
    anova_result = pg.welch_anova(data= df[(df["Profession_classification_chinese"]==Profession_classification) & ~(df["mix_salary"].isnull()) & (df["month_salary_classfication"] == 1)].copy() , dv="mix_salary", between="area")
        
    if anova_result["p-unc"].values[0] < 0.05 :
        # print(f"{Profession_classification}:{anova_result['p-unc'].values[0]}")
        sign_df = pandas.DataFrame(sign_list)
        sign_df["Welch's_anova_test"] = "pass"
        df_base_Welchs_anova_test = pandas.concat([df_base_Welchs_anova_test, sign_df])
        pass
    else :  
        sign_df = pandas.DataFrame(sign_list)
        sign_df["Welch's_anova_test"] = "reject"
        df_base_Welchs_anova_test = pandas.concat([df_base_Welchs_anova_test, sign_df])
        continue

    # 進行Games-Howell檢定
    result_county_list = []  
    posthoc_result = pairwise_gameshowell(data= df[(df["Profession_classification_chinese"]==Profession_classification) & ~(df["mix_salary"].isnull()) & (df["month_salary_classfication"] == 1)].copy() , dv="mix_salary", between="area")

    significant_pairs = posthoc_result[posthoc_result['pval'] < 0.05].copy()
    significant_pairs["pairs"] = significant_pairs["A"] + significant_pairs["B"]
    for pair in list(dict.fromkeys(significant_pairs["pairs"])) :
        if "台北市" not in pair :
            continue 
        else : 
            # 台北平均數
            Taipai_mean = df[(df["Profession_classification_chinese"]==Profession_classification) & (df["area"] == "台北市") & (df["month_salary_classfication"] == 1)]["mix_salary"].mean()
            # print(Taipai_mean)
            # 該縣平均數
            county_mean = df[(df["Profession_classification_chinese"]==Profession_classification) & (df["area"] == pair.replace("台北市", "")) & (df["month_salary_classfication"] == 1)]["mix_salary"].mean()
            # print(county_mean)
            result_county = {"縣市" : pair.replace("台北市", ""), "職業類型" :Profession_classification, "Games-Howell_test" :"pass", "平均值差異" : (county_mean - Taipai_mean)} # 縣市值 - 台北值。
            result_county_list.append(result_county)
            # print(result_county)
    result_county_df = pandas.DataFrame(result_county_list)
    df_base_Games_Howell = pandas.concat([df_base_Games_Howell, result_county_df])



In [12]:
df[(df["Profession_classification_chinese"]=="傳播藝術、設計類") & (df["area"] == "基隆市") & (df["month_salary_classfication"] == 1)]["mix_salary"].mean()

44546.653333333335

In [13]:
df[(df["Profession_classification_chinese"]=="傳播藝術、設計類") & (df["area"] == "台北市") & (df["month_salary_classfication"] == 1)]["mix_salary"].mean()

33442.88888888889

In [4]:
df_test = pandas.read_csv(os_path(["檢定資料", "同質性檢定.csv"]))
df_test = pandas.merge(df_test, df_base_Welchs_anova_test, on=["縣市", "職業類型"], how="outer")


In [5]:
df_test = pandas.merge(df_test, df_base_Games_Howell, on=["縣市", "職業類型"], how="outer")
df_test

,縣市,職業類型,是否通過_levene,是否通過_Shapiro_Wilk,能否進行檢定,Welch's_anova_test,Games-Howell_test,平均值差異
0,新北市,經營、人資類,pass,pass,pass,pass,pass,-1131.755349
1,基隆市,經營、人資類,none,pass,none,pass,pass,-2482.235873
2,桃園市,經營、人資類,none,pass,none,pass,pass,-1398.195581
3,台南市,經營、人資類,pass,pass,pass,pass,pass,-3187.900519
4,宜蘭縣,經營、人資類,pass,pass,pass,pass,pass,-3326.439279
...,...,...,...,...,...,...,...,...
355,澎湖縣,其他類,none,pass,none,pass,NaN,NaN
356,苗栗縣,其他類,pass,pass,pass,pass,pass,-2920.312351
357,嘉義縣,其他類,pass,pass,pass,pass,pass,-3154.587281
358,嘉義市,其他類,pass,pass,pass,pass,pass,-3827.646523


In [11]:
df_test["職業類型"].unique()

array(['經營、人資類', '行政、總務、法務類', '財會、金融專業類', '行銷、企劃、專案管理類', '客服、門市、業務、貿易類',
       '餐飲、旅遊、美容美髮類', '資訊軟體系統類', '操作、技術、維修類', '資材、物流、運輸類', '營建、製圖類',
       '傳播藝術、設計類', '文字、傳媒工作類', '醫療、服務保健類', '學術、教育、輔導類', '研發相關類',
       '生產製造、品管、環衛類', '軍警消、保全類', '其他類'], dtype=object)

In [6]:
df_test = df_test[~(df_test["縣市"] == "台北市")]
df_test["Games-Howell_test"] = df_test["Games-Howell_test"].fillna("reject")
df_test["平均值差異"] = df_test["平均值差異"].fillna("未達顯著水準")
df_test.to_excel(os_path(["檢定資料", "Anova_test_area_salary_month.xlsx"]), index= False)